# Submission notebook of HuBMAP - Pytorch smp Unet++ Inference


Train part is here.

https://www.kaggle.com/nayuts/hubmap-pytorch-smp-unet

--------------------------

I create train and inference notebook for Unet++ of [segmentation_models.pytorch](https://github.com/qubvel/segmentation_models.pytorch).

I use only pytorch for framework.

The accuracy of the model is going to be tuned and improved in the future.

I published this notebook for our reference as an example implementation using only pytorch.

I refered following two great notebooks for training and inference.

- https://www.kaggle.com/iafoss/hubmap-pytorch-fast-ai-starter

- https://www.kaggle.com/curiosity806/hubmap-use-catalyst-smp-and-albumentations

And refered following great notebook for dice loss.

- https://www.kaggle.com/bigironsphere/loss-function-library-keras-pytorch

## Load libraries

In [ ]:
import gc
import os
import random
import sys
import time
import warnings
warnings.simplefilter("ignore")

#import pdb
#import zipfile
#import pydicom
from albumentations import *
from albumentations.pytorch import ToTensor
import cv2
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
import tifffile as tiff
import torch
import torch.backends.cudnn as cudnn
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader, Dataset, sampler
from tqdm import tqdm_notebook as tqdm

To use segmentation_models_pytorch in offline environment, I loaded modules I cloned and uploaded to dataset.

In [ ]:
#https://www.kaggle.com/hfutybx/unet-densenet121-lung-of-segmentation/data

sys.path.append('../input/efficientnetpytorchaug252020/EfficientNet-PyTorch-master')
sys.path.append('../input/pretrainedmodels/pretrainedmodels-0.7.4/')
sys.path.append('../input/pytorchimagemodelsoct302020/pytorch-image-models-master')
sys.path.append('../input/segmentation-models-pytorch0-1-2/segmentation_models.pytorch-master')
import segmentation_models_pytorch as smp

##  Set parameters

In [ ]:
def set_seed(seed=2**3):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
set_seed(121)

To save time, the number of "folds" is smaller. 
These days, depending on the model, I think it's more common to use 5 ~ 10.

In [ ]:
fold = 0
nfolds = 3
reduce = 4
sz = 256

BATCH_SIZE = 16
DEVICE = ('cuda' if torch.cuda.is_available() else 'cpu')
NUM_WORKERS = 4
SEED = 2020
TH = 0.50  #threshold for positive predictions

DATA = '../input/hubmap-kidney-segmentation/test/'
LABELS = '../input/hubmap-kidney-segmentation/train.csv'
MASKS = '../input/hubmap-256x256/masks/'
TRAIN = '../input/hubmap-256x256/train/'
df_sample = pd.read_csv('../input/hubmap-kidney-segmentation/sample_submission.csv')

## Util functions

In [ ]:
#https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
#with bug fix
def rle_encode_less_memory(img):
    #watch out for the bug
    pixels = img.T.flatten()
    
    # This simplified method requires first and last pixel to be zero
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] -= runs[::2]
    
    return ' '.join(str(x) for x in runs)

## Dataset

In [ ]:
def get_aug(p=1.0):
    return Compose([
        HorizontalFlip(),
        VerticalFlip(),
        RandomRotate90(),
        ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.2, rotate_limit=15, p=0.9, 
                         border_mode=cv2.BORDER_REFLECT),
        OneOf([
            OpticalDistortion(p=0.3),
            GridDistortion(p=.1),
            IAAPiecewiseAffine(p=0.3),
        ], p=0.3),
        OneOf([
            HueSaturationValue(10,15,10),
            CLAHE(clip_limit=2),
            RandomBrightnessContrast(),            
        ], p=0.3),
    ], p=p)

## Model

In [ ]:
def get_UnetPlusPlus():
    model =  smp.UnetPlusPlus(
                 encoder_name='efficientnet-b3',
                 encoder_weights=None,
                 in_channels=3,
                 classes=1)
    return model

# Inference

In [ ]:
# https://www.kaggle.com/iafoss/256x256-images
mean = np.array([0.65459856,0.48386562,0.69428385])
std = np.array([0.15167958,0.23584107,0.13146145])

def img2tensor(img,dtype:np.dtype=np.float32):
    if img.ndim==2 : img = np.expand_dims(img,2)
    img = np.transpose(img,(2,0,1))
    return torch.from_numpy(img.astype(dtype, copy=False))

class HuBMAPTestDataset(Dataset):
    def __init__(self, imgs, idxs):
        self.imgs = imgs
        self.fnames = idxs
        
    def __len__(self):
        return len(self.fnames)
    
    def __getitem__(self, idx):
        return img2tensor((self.imgs[idx]/255.0 - mean)/std)

In [ ]:
models = []

"""for fold in range(nfolds):

    model = get_UnetPlusPlus().to(DEVICE)
    model.load_state_dict(torch.load(f"../input/hubmap-pytorch-smp-unet/FOLD{fold}_.pth"))
    models.append(model)"""
model = smp.Unet('se_resnext50_32x4d', encoder_weights=None, classes=1).cuda()
model.load_state_dict(torch.load("../input/testngerror/FOLD-2-model.pth"))
models.append(model)


In [ ]:
#iterator like wrapper that returns predicted masks
class Model_pred:
    def __init__(self, models, dl, tta:bool=True, half:bool=False):
        self.models = models
        self.dl = dl
        self.tta = tta
        self.half = half
        
    def __iter__(self):
        count=0
        with torch.no_grad():
            for x in self.dl: #iter(self.dl):
                x = x.to(DEVICE)
                if self.half: x = x.half()
                py = None
                for model in self.models:
                    p = model(x)
                    p = torch.sigmoid(p).detach()
                    if py is None: py = p
                    else: py += p
                if self.tta:
                    #x,y,xy flips as TTA
                    flips = [[-1],[-2],[-2,-1]]
                    for f in flips:
                        xf = torch.flip(x,f)
                        for model in self.models:
                            p = model(xf)
                            p = torch.flip(p,f)
                            py += torch.sigmoid(p).detach()
                    py /= (1+len(flips))        
                py /= len(self.models)
                    
                py = F.upsample(py, scale_factor=reduce, mode="bilinear")
                py = py.permute(0,2,3,1).float().cpu()
                batch_size = len(py)
                for i in range(batch_size):
                    yield py[i]
                    count += 1
                    
    def __len__(self):
        return len(self.dl.dataset)

In [ ]:
#Somehow I cannot resolve the submission error with consideration of the
#private LB data, and the submission error doesn't give an informative
#output. So, for now I share the notbook that makes a submission only
#to the public LB, and later I'll try to resolve the issue.
#IMPORTANT: This notebook doesn't perform predictions for the private LB.
names,preds = [],[]
samples = ['b9a3865fc','b2dc8411c','26dc41664','c68fe75ea','afa5e8098']
samples_n = [id for id in df_sample.id if id not in samples]

names += samples_n
preds += [np.NaN]*len(samples_n)
df_sample = df_sample.loc[df_sample.id.isin(samples)]

In [ ]:
#https://www.kaggle.com/iafoss/256x256-images
s_th = 40  #saturation blancking threshold
p_th = 200*sz//256 #threshold for the minimum number of pixels
#names,preds = [],[]
for idx,row in tqdm(df_sample.iterrows(),total=len(df_sample)):
    idx = row['id']
    #read image
    img = tiff.imread(os.path.join(DATA,idx+'.tiff'))
    if len(img.shape) == 5: img = np.transpose(img.squeeze(), (1,2,0))
    
    #add padding to make the image dividable into tiles
    img_shape = img.shape
    pad0 = (reduce*sz - img_shape[0]%(reduce*sz))%(reduce*sz)
    pad1 = (reduce*sz - img_shape[1]%(reduce*sz))%(reduce*sz)
    img = np.pad(img,[[pad0//2,pad0-pad0//2],[pad1//2,pad1-pad1//2],[0,0]],
                 constant_values=0)

    #split image into tiles using the reshape+transpose trick
    if reduce != 1:
        img = cv2.resize(img,(img.shape[1]//reduce,img.shape[0]//reduce),
                     interpolation = cv2.INTER_AREA)
    img_shape_p = img.shape
    img = img.reshape(img.shape[0]//sz,sz,img.shape[1]//sz,sz,3)
    img = img.transpose(0,2,1,3,4).reshape(-1,sz,sz,3)

    #select tiles for running the model
    imgs,idxs = [],[]
    for i,im in enumerate(img):
        #remove black or gray images based on saturation check
        hsv = cv2.cvtColor(im, cv2.COLOR_BGR2HSV)
        h, s, v = cv2.split(hsv)
        if (s>s_th).sum() <= p_th or im.sum() <= p_th: continue
        imgs.append(im)
        idxs.append(i)
    #tile dataset
    ds = HuBMAPTestDataset(imgs,idxs)
    dl = DataLoader(ds,batch_size=BATCH_SIZE, shuffle=False)
    #dl = DataLoader(ds,BATCH_SIZE,num_workers=NUM_WORKERS,shuffle=False,pin_memory=True)
    mp = Model_pred(models,dl)
    
    #generate masks
    mask = torch.zeros(img.shape[0],sz*reduce,sz*reduce,dtype=torch.int8)
    for i,p in zip(idxs,iter(mp)): mask[i] = p.squeeze(-1) > TH
    
    #reshape tiled masks into a single mask and crop padding
    mask = mask.view(img_shape_p[0]//sz,img_shape_p[1]//sz,sz*reduce,sz*reduce).\
        permute(0,2,1,3).reshape(img_shape_p[0]*reduce,img_shape_p[1]*reduce)
    mask = mask[pad0//2:-(pad0-pad0//2) if pad0 > 0 else img_shape_p[0]*reduce,
        pad1//2:-(pad1-pad1//2) if pad1 > 0 else img_shape_p[1]*reduce]
    
    #convert to rle
    #https://www.kaggle.com/bguberfain/memory-aware-rle-encoding
    rle = rle_encode_less_memory(mask.numpy())
    names.append(idx)
    preds.append(rle)
    gc.collect()

In [ ]:
df = pd.DataFrame({'id':names,'predicted':preds})
df.to_csv('submission.csv',index=False)